# Introduction

This file creates the (a) Boathire dataset. Each Boathire dataset contains the same process variants, but each dataset is unique, since the natural language events are created randomly.

In [1]:
# Initialise Open AI client
from openai import OpenAI
client = OpenAI()

## Variants 1-9: Large Group requests
There are 54 traces in total to generate.
Start with Receive Enquiry / Uncatered / Large Group activity.
This could be done faster in a bulk request, but we're not doing that to avoid running into any rate limits.

In [115]:
def createReceiveEnquiry(request):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": request
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

receiveEnquiry = dict()
for i in range(54):
  receiveEnquiry[i] = createReceiveEnquiry("You want to hire a boat for a party (no catering required - do not mention it) of between 30 and 50 people (pick a number). Make up the details (e.g. what event, how many guests, date, time, your name, etc.)"


Now create 54 Send Offer - Pier Pressure events. Note that it's not garantueed to get an offer for the Pier Pressure, but this is intentional to add some variety to the dataset.

In [119]:
def createSendOfferEvent(receiveEnquiry):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": "You are James and are working for a British boat hire company called Boathire. Boathire Boathire has three crewed charter boats for private events cruises: The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering. \nBoathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer."
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": receiveEnquiry
            }
        ]
        }
    ],
    temperature=1,
    max_tokens=1600,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content

sendOffer = dict()
for i in range(len(receiveEnquiry)):
    sendOffer[i] = createSendOfferEvent(receiveEnquiry[i])

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[1].content[0].text': expected a string, but got an integer instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content[0].text', 'code': 'invalid_type'}}

Alright, we got our RE and SO events for the first 54 traces. Now let's do:
## Variant 1 RE - SO - Receive Booking

In [ ]:
def createReceiveBooking(re, so, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Confirm the booking of the " + boat + "."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

event3 = dict()
for i in range(0, 16):
  event3[i] = createReceiveBooking(receiveEnquiry[i], sendOffer[i], "Pier Pressure")

## Initialise event log df and add variant 1

In [ ]:
import pandas as pd
eventLog = pd.DataFrame()
for caseId in range(0,16):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(1,4),
            'Text': [receiveEnquiry[caseId], sendOffer[caseId], event3[caseId]],
            'VariantId': 1,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Receive Booking'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure', 'Receive Booking / Crewed / Pier Pressure']
        }
    )    
    eventLog = pd.concat([eventLog, df])

eventLog

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,0,1,Subject: Boat Hire Inquiry for Corporate Event...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,0,2,Subject: Re: Boat Hire Inquiry for Corporate E...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,0,3,"Subject: Confirming Booking for ""The Pier Pres...",1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,1,1,Subject: Boat Hire Inquiry for Private Party o...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,1,2,Subject: Re: Boat Hire Inquiry for Private Par...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,1,3,Subject: Confirmation of Booking for 'Pier Pre...,1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,2,1,Subject: Inquiry for Boat Hire for Corporate E...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,2,2,Subject: Re: Inquiry for Boat Hire for Corpora...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,2,3,Subject: Confirmation of Booking for The Pier ...,1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,3,1,Subject: Inquiry on Boat Availability for an E...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group


## Variant 2: Reject offer
There will be 13 variants where the offer will be rejected.

In [ ]:
def createOfferRejected(re, so):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            #"text": "Decline/reject the offer. Make up a reason, if you want."
            "text": "Decline/reject the offer."
          }
        ]
      }
    ],
    temperature=1.3,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#createOfferRejected(receiveEnquiry[16], sendOffer[16])
for i in range(16, 16+13):
  event3[i] = createOfferRejected(receiveEnquiry[i], sendOffer[i])


Add variant 2 to event log

In [ ]:
for caseId in range(16, 16+13):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(1,4),
            'Text': [receiveEnquiry[caseId], sendOffer[caseId], event3[caseId]],
            'VariantId': 2,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Offer Rejected'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure', 'Offer Rejected / Crewed / Pier Pressure']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,0,1,Subject: Boat Hire Inquiry for Corporate Event...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,0,2,Subject: Re: Boat Hire Inquiry for Corporate E...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,0,3,"Subject: Confirming Booking for ""The Pier Pres...",1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,1,1,Subject: Boat Hire Inquiry for Private Party o...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,1,2,Subject: Re: Boat Hire Inquiry for Private Par...,1,Send Offer,Send Offer / Crewed / Pier Pressure
...,...,...,...,...,...,...
1,27,2,Subject: Re: Boat Hire for Anniversary Celebra...,2,Send Offer,Send Offer / Crewed / Pier Pressure
2,27,3,Subject: Re: Boat Hire for Anniversary Celebra...,2,Offer Rejected,Offer Rejected / Crewed / Pier Pressure
0,28,1,Subject: Boat Hire Inquiry for Private Celebra...,2,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,28,2,Subject: Re: Boat Hire Inquiry for Private Cel...,2,Send Offer,Send Offer / Crewed / Pier Pressure


Add variant 3, which doesn't have an event 3

In [ ]:
for caseId in range(16+13, 16+13+13):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(2),
            'Text': [receiveEnquiry[caseId], sendOffer[caseId]],
            'VariantId': 3,
            'Activity': ['Receive Enquiry', 'Send Offer'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,0,1,Subject: Boat Hire Inquiry for Corporate Event...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,0,2,Subject: Re: Boat Hire Inquiry for Corporate E...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,0,3,"Subject: Confirming Booking for ""The Pier Pres...",1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,1,1,Subject: Boat Hire Inquiry for Private Party o...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,1,2,Subject: Re: Boat Hire Inquiry for Private Par...,1,Send Offer,Send Offer / Crewed / Pier Pressure
...,...,...,...,...,...,...
1,39,1,Subject: Re: Inquiry About Boat Hire for Priva...,3,Send Offer,Send Offer / Crewed / Pier Pressure
0,40,0,Subject: Inquiry: Boat Hire for Private Event ...,3,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,40,1,Subject: Re: Inquiry: Boat Hire for Private Ev...,3,Send Offer,Send Offer / Crewed / Pier Pressure
0,41,0,Subject: Inquiry for Boat Hire on November 15t...,3,Receive Enquiry,Receive Enquiry / Uncatered / Large Group


## Variants 4-9 - Discount requested

In [ ]:
def createDiscountRequested(re, so):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Ask for a discount."
          }
        ]
      }
    ],
    temperature=1.3,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#createDiscountRequested(sendOffer[42], receiveEnquiry[42])
for i in range(42, 42+12):
  event3[i] = createDiscountRequested(sendOffer[i], receiveEnquiry[i])

Add Request Approval emails

In [ ]:
def createRequestApproval(re, so, dr):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are James and are working for a British boat hire company called Boathire. 
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            
            When customer request discounts, this needs to be handled by your supervisor Sarah."""
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Write a quick email to your supervisor Sarah, asking her to get in contact with the customer and offer or decline the requested discount."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#createRequestApproval(sendOffer[42], receiveEnquiry[42], event3[42])
requestApproval = dict()
for i in range(42, 42+12):
  requestApproval[i] = createDiscountRequested(sendOffer[i], receiveEnquiry[i])
#requestApproval = [createDiscountRequested(sendOffer[i], receiveEnquiry[i]) for i in range(42, 42+12)]

Sarah is sending revised offers, with some discount.

In [ ]:
def createDiscountedOffer(re, so, dr, ra):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are Sarah and are working for a British boat hire company called Boathire. 
            You are James' supervisor.
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            """
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Offer a discount to the customer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#createDiscountedOffer(sendOffer[42], receiveEnquiry[42], event3[42], requestApproval[42])
event5 = dict()
for i in range(42, 42+7):
  event5[i] = createDiscountRequested(sendOffer[i], receiveEnquiry[i])

Now decline the rest of the offers...

In [ ]:
def createDeclineDiscount(re, so, dr, ra):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": """You are Sarah and are working for a British boat hire company called Boathire. 
            You are James' supervisor.
            Boathire has three crewed charter boats for private events cruises: 
            The Pier Pressure, for up to 60 people, the Aquaholic, for up to 30 people and the Harbor Harmony. Each boat can be chartered with or without catering.
            
            Boathire also has a small fleet of sailing boats and rowing boats. The available sailing boats are Delphia 24 and Wayfairer.
            """
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Regretfully decline the discount request of the customer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#createDeclineDiscount(sendOffer[49], receiveEnquiry[49], event3[49], requestApproval[49])
#event5 = dict()
for i in range(49, 49+5):
  event5[i] = createDeclineDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i])

Now create the Receive booking responses from the client

In [ ]:
def createReceiveBookingAfterDiscount(re, so, dr, ra, e5, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": e5
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Confirm the booking of the " + boat + "."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#i = 42
#createReceiveBookingAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")

event6 = dict()
for i in range(42, 42+5):
  event6[i] = createReceiveBookingAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")


for i in range(49, 49+2):
  event6[i] = createReceiveBookingAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")


No do the decline offer events

In [ ]:
def createDeclineOfferAfterDiscount(re, so, dr, ra, e5, boat):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": "You are a British customer looking to hire a boat and are sending emails to a boat hire company called Boathire. You do not include placeholder text in your answers."
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": re
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": so
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": dr
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": ra
          }
        ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": e5
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Decline/reject the offer."
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=3020,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n = 1
  )
  return response.choices[0].message.content

#i = 47
#createDeclineOfferAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")

for i in range(47, 47+1):
  event6[i] = createDeclineOfferAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")


for i in range(51, 51+1):
  event6[i] = createDeclineOfferAfterDiscount(sendOffer[i], receiveEnquiry[i], event3[i], requestApproval[i], event5[i], "Pier Pressure")


Add it to the eventlog (variants 4-9):

In [ ]:
# variant 4
for caseId in range(42, 42+5):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(6),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId], 
                     event6[caseId]],
            'VariantId': 3,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Receive Booking'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Send offer with Discount / Crewed / Pier Pressure', 'Receive Booking / Crewed / Pier Pressure / Discounted']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,0,1,Subject: Boat Hire Inquiry for Corporate Event...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,0,2,Subject: Re: Boat Hire Inquiry for Corporate E...,1,Send Offer,Send Offer / Crewed / Pier Pressure
2,0,3,"Subject: Confirming Booking for ""The Pier Pres...",1,Receive Booking,Receive Booking / Crewed / Pier Pressure
0,1,1,Subject: Boat Hire Inquiry for Private Party o...,1,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,1,2,Subject: Re: Boat Hire Inquiry for Private Par...,1,Send Offer,Send Offer / Crewed / Pier Pressure
...,...,...,...,...,...,...
1,46,1,"Dear Emily,\n\nThank you for reaching out to B...",3,Send Offer,Send Offer / Crewed / Pier Pressure
2,46,2,Subject: Inquiry for Boat Hire on 15th Novembe...,3,Discount Requested,Discount Requested
3,46,3,Subject: Inquiry for Boat Hire on 15th Novembe...,3,Request Approval,Request Approval
4,46,4,Subject: Inquiry and Possible Discount for Boa...,3,Send offer with Discount,Send offer with Discount / Crewed / Pier Pressure


In [ ]:
#variant 5
for caseId in range(47, 47+1):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(6),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId], 
                     event6[caseId]],
            'VariantId': 5,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount', 'Offer rejected'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Send offer with Discount / Crewed / Pier Pressure', 'Offer rejected / Crewed / Pier Pressure / Discounted']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog.tail()

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
1,47,1,Subject: Re: Boat Hire Inquiry for a Corporate...,3,Send Offer,Send Offer / Crewed / Pier Pressure
2,47,2,Subject: Boat Hire Inquiry and Discount Reques...,3,Discount Requested,Discount Requested
3,47,3,Subject: Boat Hire Inquiry for a Corporate Gat...,3,Request Approval,Request Approval
4,47,4,Subject: Re: Boat Hire Inquiry for a Corporate...,3,Send offer with Discount,Send offer with Discount / Crewed / Pier Pressure
5,47,5,Subject: Re: Boat Hire Inquiry for a Corporate...,3,Offer rejected,Offer rejected / Crewed / Pier Pressure / Disc...


In [ ]:
# variant 6
for caseId in range(48, 48+1):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(5),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId]],
            'VariantId': 6,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Send offer with Discount'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Send offer with Discount / Crewed / Pier Pressure']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog.tail()

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,48,0,Subject: Boat Hire Inquiry for Corporate Event...,3,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,48,1,Subject: Re: Boat Hire Inquiry for Corporate E...,3,Send Offer,Send Offer / Crewed / Pier Pressure
2,48,2,Subject: Re: Boat Hire Inquiry for Corporate E...,3,Discount Requested,Discount Requested
3,48,3,Subject: Re: Boat Hire Inquiry for Corporate E...,3,Request Approval,Request Approval
4,48,4,Subject: Re: Boat Hire Inquiry for Corporate E...,3,Send offer with Discount,Send offer with Discount / Crewed / Pier Pressure


In [ ]:
# variant 7
for caseId in range(49, 49+2):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(6),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId],
                     event6[caseId]],
            'VariantId': 7,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Receive Booking'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Decline Discount / Crewed / Pier Pressure', 'Receive Booking / Crewed / Pier Pressure / Undiscounted']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog.tail()

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
1,50,1,Subject: Re: Inquiry for Boat Hire on 15th Nov...,7,Send Offer,Send Offer / Crewed / Pier Pressure
2,50,2,Subject: Re: Inquiry for Boat Hire on 15th Nov...,7,Discount Requested,Discount Requested
3,50,3,Subject: Re: Inquiry for Boat Hire on 15th Nov...,7,Request Approval,Request Approval
4,50,4,Subject: Re: Inquiry for Boat Hire on 15th Nov...,7,Decline Discount,Decline Discount / Crewed / Pier Pressure
5,50,5,Subject: Confirmation of Booking for The Pier ...,7,Receive Booking,Receive Booking / Crewed / Pier Pressure / Und...


In [ ]:
# variant 8
for caseId in range(51, 51+1):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(6),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId],
                     event6[caseId]],
            'VariantId': 8,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount', 'Offer rejected'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Decline Discount / Crewed / Pier Pressure', 'Offer rejected / Crewed / Pier Pressure / Undiscounted']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog.tail()

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
1,51,1,Subject: Re: Boat Hire Inquiry for Private Par...,8,Send Offer,Send Offer / Crewed / Pier Pressure
2,51,2,Subject: Boat Hire Inquiry for Private Party o...,8,Discount Requested,Discount Requested
3,51,3,Subject: Re: Boat Hire Inquiry for Private Par...,8,Request Approval,Request Approval
4,51,4,Subject: Re: Boat Hire Inquiry for Private Par...,8,Decline Discount,Decline Discount / Crewed / Pier Pressure
5,51,5,Subject: Re: Boat Hire Inquiry for Private Par...,8,Offer rejected,Offer rejected / Crewed / Pier Pressure / Undi...


In [ ]:
# variant 9
for caseId in range(52, 52+2):
    df = pd.DataFrame(
        {
            'CaseId': caseId,
            'ActivityId': range(5),
            'Text': [receiveEnquiry[caseId], 
                     sendOffer[caseId], 
                     event3[caseId], 
                     requestApproval[caseId], 
                     event5[caseId]],
            'VariantId': 8,
            'Activity': ['Receive Enquiry', 'Send Offer', 'Discount Requested', 'Request Approval', 'Decline Discount'],
            'SpecificActivity': ['Receive Enquiry / Uncatered / Large Group', 'Send Offer / Crewed / Pier Pressure',
                'Discount Requested', 'Request Approval', 'Decline Discount / Crewed / Pier Pressure']
        }
    )    
    eventLog = pd.concat([eventLog, df])
eventLog.tail()

,CaseId,ActivityId,Text,VariantId,Activity,SpecificActivity
0,53,0,Subject: Inquiry about Boat Hire for Corporate...,8,Receive Enquiry,Receive Enquiry / Uncatered / Large Group
1,53,1,Subject: Re: Inquiry about Boat Hire for Corpo...,8,Send Offer,Send Offer / Crewed / Pier Pressure
2,53,2,Subject: Inquiry about Boat Hire for Corporate...,8,Discount Requested,Discount Requested
3,53,3,Subject: Inquiry about Boat Hire for Corporate...,8,Request Approval,Request Approval
4,53,4,Subject: Re: Inquiry about Boat Hire for Corpo...,8,Decline Discount,Decline Discount / Crewed / Pier Pressure


## Variants 10-18

In [ ]:
# Receive Enquiry
for i in range(54, 54+9):
    receiveEnquiry[i] = createReceiveEnquiry(
        """You want to hire a boat for a party (no catering required - do not mention it) of between 10 and 30 people (pick a number). 
            Make up the details (e.g. what event, how many guests, date, time, your name, etc.)""")


In [ ]:
for i in range(54, 54+9):
    sendOffer[i] = createSendOfferEvent(receiveEnquiry[i])

In [ ]:
test_list = [0, 1,2,3]
test_list[4] = 4

IndexError: list assignment index out of range